In [2]:
import os
from pathlib import Path
from typing import List
import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI
import minsearch as ms

In [3]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [4]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    results_dict = {}
    for q in tqdm(ground_truth):
        try:
            doc_id = q['id']
            results = search_function(q)

            relevance = [str(d['id']) == str(doc_id) for d in results]

            results_dict[q['id']] = (q, results)
            relevance_total.append(relevance)

        except Exception as e:
            print(f"Error processing query: {q} with exeption: {e}")

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [5]:
DATA_PATH = os.getenv("DATA_PATH", "../DATASETS/faq_sacmex.csv")

def ingest_data(file_path: Path = Path(DATA_PATH), text_fields: List[str] = None):
    # Lee el archivo csv
    df = pd.read_csv(file_path)

    print(df.head())

    # Convierte los campos a string
    if text_fields:
        for field in text_fields:
            if field in df.columns:
                df[field] = df[field].astype(str)

    # Convierte el DataFrame a diccionario
    return df.to_dict(orient='records')


## Evaluate retrieval using Elastic Search and the ground truth data

In [6]:
df_question = pd.read_csv('../DATASETS/ground-truth-retrieval_llama3_2_3b.csv')
df_question.head()

,id,pregunta
0,d7c4ce5eda85cd602edc71a3f29193e0,¿Dónde puedo reportar una fuga de agua?
1,c6b1388a0227cae6d6045a1dc7dd82c5,¿Qué debo hacer si veo una fuga de agua en la ...
2,b9e493259e0b9fb9f94e4f4d66ab8ded,¿Qué debo hacer si veo una fuga de agua en mi ...
3,5229ec6562607f2de830a7826a099964,¿Qué debo hacer si mi consumo de agua parece h...
4,9055d52c36234595f93430ca60610b82,¿Cuánto tiempo tarda en repararse una fuga de ...


In [7]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?'}

In [8]:
documents = ingest_data()
documents[0]

                                 id  \
0  d7c4ce5eda85cd602edc71a3f29193e0   
1  c6b1388a0227cae6d6045a1dc7dd82c5   
2  b9e493259e0b9fb9f94e4f4d66ab8ded   
3  5229ec6562607f2de830a7826a099964   
4  9055d52c36234595f93430ca60610b82   

                                            pregunta  \
0            ¿Dónde puedo reportar una fuga de agua?   
1  ¿Qué debo hacer si veo una fuga de agua en la ...   
2  ¿Qué debo hacer si veo una fuga de agua en mi ...   
3  ¿Qué debo hacer si mi consumo de agua parece h...   
4  ¿Cuánto tiempo tarda en repararse una fuga de ...   

                                           respuesta    document  
0  Debes reportarla al organismo operador de agua...  faq_sacmex  
1  En la Ciudad de México, debes reportarla al 55...  faq_sacmex  
2  Lo primero que debes hacer es cortar el sumini...  faq_sacmex  
3  Si no tienes fuga de agua en tu casa, puedes t...  faq_sacmex  
4  Después de supervisar los materiales, maquinar...  faq_sacmex  


{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?',
 'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
 'document': 'faq_sacmex'}

## Create Embeddings using Sentence Transformer

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

In [10]:
len(model.encode("Ejemplo encode de texto"))

768

In [11]:
# Created the dense vector using the pre-trained model
operations = []
embeddings = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["respuesta_vector"] = model.encode(doc["respuesta"]).tolist()
    operations.append(doc)
    embeddings.append(doc["respuesta_vector"])

  0%|          | 0/11 [00:00<?, ?it/s]

In [12]:
len(operations[0].get("respuesta_vector"))

768

## Step 3: Setup ElasticSearch connection


In [38]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '93df874dead8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'xnEBS8X-Rr-ZXdf_IeXFnQ', 'version': {'number': '8.15.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018', 'build_date': '2024-10-09T22:08:00.328917561Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties":
            {
                'id': {"type": "keyword"},
                'pregunta': {"type": "text"},
                'respuesta': {"type": "text"},
                'document': {"type": "text"},
                'respuesta_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
            }
    }
}


In [15]:
index_name = "faq_sacmex"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_sacmex'})

### Step 5: Add documents into index


In [16]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/11 [00:00<?, ?it/s]

### Step 6: Create end user query


In [17]:
search_term = "¿Como reporto una fuga en mi unidad?."
vector_search_term = model.encode(search_term)

In [18]:
query = {
    "field": "respuesta_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10,
}

In [19]:
res = es_client.search(index=index_name, knn=query,
                    source=['pregunta', 'respuesta', 'document', 'respuesta_vector'])
res["hits"]["hits"]

[{'_index': 'faq_sacmex',
  '_id': 'iBHidpMBeVQVR_An2iyv',
  '_score': 0.8100641,
  '_source': {'pregunta': '¿Qué debo tener en cuenta para levantar mi reporte?',
   'respuesta': 'Para levantar un reporte debes tener en cuenta lo siguiente:\n- Contar con la dirección exacta del lugar y la descripción precisa del problema\n- Proporcionar nombre de la alcaldia, colonia, calle y número oficial, calles aledañas, nombre del reportante y número telefónico.',
   'document': 'faq_sacmex',
   'respuesta_vector': [-0.011219686828553677,
    -0.008590794168412685,
    -0.024403022602200508,
    0.004356065299361944,
    -0.015704484656453133,
    0.010333926416933537,
    0.0019830616656690836,
    0.02669011615216732,
    -0.006960637401789427,
    -0.021611327305436134,
    0.01873764395713806,
    0.0006045747431926429,
    0.05877666920423508,
    -0.006333488505333662,
    -0.008793406188488007,
    -0.04880858585238457,
    -0.016167573630809784,
    0.03279992565512657,
    -0.063594855368

### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)


In [24]:
knn_query = {
    "field": "respuesta_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [25]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"document": "faq_sacmex"},
    },
    knn=knn_query,
    size=5
)

In [26]:
response["hits"]["hits"]

[{'_index': 'faq_sacmex',
  '_id': 'ZxGlc5MBeVQVR_AnryyL',
  '_score': 0.8526237,
  '_source': {'id': '977ff53e8650679587f99ee537c8b095',
   'pregunta': '¿Qué debo tener en cuenta para levantar mi reporte?',
   'respuesta': 'Para levantar un reporte debes tener en cuenta lo siguiente:\n- Contar con la dirección exacta del lugar y la descripción precisa del problema\n- Proporcionar nombre de la alcaldia, colonia, calle y número oficial, calles aledañas, nombre del reportante y número telefónico.',
   'document': 'faq_sacmex',
   'respuesta_vector': [-0.011219686828553677,
    -0.008590794168412685,
    -0.024403022602200508,
    0.004356065299361944,
    -0.015704484656453133,
    0.010333926416933537,
    0.0019830616656690836,
    0.02669011615216732,
    -0.006960637401789427,
    -0.021611327305436134,
    0.01873764395713806,
    0.0006045747431926429,
    0.05877666920423508,
    -0.006333488505333662,
    -0.008793406188488007,
    -0.04880858585238457,
    -0.016167573630809784,

### Evaluate retrieval using Elastic Search and the ground truth data generated with llama3.2:3b

In [20]:
import numpy as np


class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [21]:
X = np.array(embeddings)
X.shape

(11, 768)

In [22]:
v = model.encode("¿Cuales son los telefonos de la SACMEX?")

In [23]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'id': '221e2b757cee67a9babd822666eae6b1',
  'pregunta': '¿Cómo reportar fugas de agua en CDMX?',
  'respuesta': 'Para realizar cualquier reporte por fuga en la vía pública, el SACMEX pone a disposición de la ciudadanía el número telefónico de LOCATEL *0311 y 55 5658 1111, así como sus redes sociales oficiales @SacmexCDMX en X (antes Twitter) y @SistemaDeAguasCDMX en',
  'document': 'faq_sacmex',
  'respuesta_vector': [-0.03254207968711853,
   -0.0006378046236932278,
   -0.008281011134386063,
   0.022378575056791306,
   0.028625616803765297,
   -0.022966161370277405,
   0.02649880200624466,
   0.012834921479225159,
   0.01753140799701214,
   -0.00525951711460948,
   -0.03462344408035278,
   0.025997750461101532,
   0.02246468886733055,
   0.019036276265978813,
   -0.04308563098311424,
   0.038950372487306595,
   -0.014198190532624722,
   -0.014872013591229916,
   -0.04596209153532982,
   -0.0023023125249892473,
   -0.02717003971338272,
   0.01872623898088932,
   0.016488680616021156,


In [55]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function, field):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q, field)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [25]:
def numpy_cosine_search(q):
    question = q['pregunta']

    v_q = model.encode(question)

    return search_engine.search(v_q, num_results=5)

In [26]:
from tqdm import tqdm

evaluate(ground_truth, numpy_cosine_search)

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 19.14it/s]


{'hit_rate': 0.8181818181818182, 'mrr': 0.4606060606060607}

### Approach 2


In [41]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties":
            {
                'id': {"type": "keyword"},
                'pregunta': {"type": "text"},
                'respuesta': {"type": "text"},
                'document': {"type": "text"},
                'pregunta_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
                'respuesta_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
                'preg_resp_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
            }
    }
}

index_name = "faq_sacmex"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_sacmex'})

In [42]:
operations = []
embeddings = []
for doc in tqdm(documents):
    doc["pregunta_vector"] = model.encode(doc["pregunta"]).tolist()
    doc["respuesta_vector"] = model.encode(doc["respuesta"]).tolist()
    preg_resp = f"{doc["pregunta"]} {doc["respuesta"]}"
    doc["preg_resp_vector"] = model.encode(preg_resp).tolist()
    operations.append(doc)
    embeddings.append((doc["pregunta_vector"], doc["respuesta_vector"], doc["preg_resp_vector"]))

100%|██████████| 11/11 [00:03<00:00,  3.54it/s]


In [44]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|██████████| 11/11 [00:00<00:00, 42.31it/s]


In [43]:
len(documents[0]['pregunta_vector']), len(documents[0]['respuesta_vector']), len(documents[0]['preg_resp_vector'])

(768, 768, 768)

In [46]:
def elastic_search_knn(field, vector, filter=None):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 100
    }

    search_query = {
        "knn": knn,
        "_source": ["pregunta", "respuesta", "id"]

    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [47]:
elastic_search_knn('respuesta_vector', v)

[{'id': '221e2b757cee67a9babd822666eae6b1',
  'pregunta': '¿Cómo reportar fugas de agua en CDMX?',
  'respuesta': 'Para realizar cualquier reporte por fuga en la vía pública, el SACMEX pone a disposición de la ciudadanía el número telefónico de LOCATEL *0311 y 55 5658 1111, así como sus redes sociales oficiales @SacmexCDMX en X (antes Twitter) y @SistemaDeAguasCDMX en'},
 {'id': '977ff53e8650679587f99ee537c8b095',
  'pregunta': '¿Qué debo tener en cuenta para levantar mi reporte?',
  'respuesta': 'Para levantar un reporte debes tener en cuenta lo siguiente:\n- Contar con la dirección exacta del lugar y la descripción precisa del problema\n- Proporcionar nombre de la alcaldia, colonia, calle y número oficial, calles aledañas, nombre del reportante y número telefónico.'},
 {'id': 'e118293c4450a72349aa684baa54f5eb',
  'pregunta': '¿Puedo reportar fugas en unidades habitacionales?',
  'respuesta': 'No, las fugas en unidades habitacionales son reparadas por la administración de las mismas u

In [48]:
elastic_search_knn('pregunta_vector', v)

[{'id': 'deb8c6e4cf607cabc9fd72178b45daae',
  'pregunta': '¿Qué otro de reportes puedo realizar?',
  'respuesta': 'Tambien puedes reportar:\n- Brotes de aguas negras\n- Tomas clandestinas\n- Mala calidad del recurso\n- Robo o desperfecto de tapas de coladeras\n- Falta de tapas de válvulas\n- Desbordes en tanques de almacenamiento'},
 {'id': 'c6b1388a0227cae6d6045a1dc7dd82c5',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en la calle?',
  'respuesta': 'En la Ciudad de México, debes reportarla al 55 5654 3210 para que una brigada del Sistema de Aguas de la Ciudad de México acuda a repararla.'},
 {'id': '977ff53e8650679587f99ee537c8b095',
  'pregunta': '¿Qué debo tener en cuenta para levantar mi reporte?',
  'respuesta': 'Para levantar un reporte debes tener en cuenta lo siguiente:\n- Contar con la dirección exacta del lugar y la descripción precisa del problema\n- Proporcionar nombre de la alcaldia, colonia, calle y número oficial, calles aledañas, nombre del reportante y número

In [49]:
elastic_search_knn('preg_resp_vector', v)

[{'id': 'c6b1388a0227cae6d6045a1dc7dd82c5',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en la calle?',
  'respuesta': 'En la Ciudad de México, debes reportarla al 55 5654 3210 para que una brigada del Sistema de Aguas de la Ciudad de México acuda a repararla.'},
 {'id': '221e2b757cee67a9babd822666eae6b1',
  'pregunta': '¿Cómo reportar fugas de agua en CDMX?',
  'respuesta': 'Para realizar cualquier reporte por fuga en la vía pública, el SACMEX pone a disposición de la ciudadanía el número telefónico de LOCATEL *0311 y 55 5658 1111, así como sus redes sociales oficiales @SacmexCDMX en X (antes Twitter) y @SistemaDeAguasCDMX en'},
 {'id': 'e118293c4450a72349aa684baa54f5eb',
  'pregunta': '¿Puedo reportar fugas en unidades habitacionales?',
  'respuesta': 'No, las fugas en unidades habitacionales son reparadas por la administración de las mismas unidades.\nEn caso de fugas exteriores a tu unidad reporta a los telefonos 5556581111 o 5556543210.'},
 {'id': '977ff53e8650679587f99e

In [50]:
def question_vector_knn(q, field):
    question = q['pregunta']
    # document = 'faq'

    v_q = model.encode(question)

    return elastic_search_knn(field, v_q)

In [51]:
question_vector_knn(ground_truth[10], 'respuesta_vector')

[{'id': 'b9e493259e0b9fb9f94e4f4d66ab8ded',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en mi casa?',
  'respuesta': 'Lo primero que debes hacer es cortar el suministro de agua, ya sea con el grifo de cierre o la válvula de cierre, que generalmente se encuentran en el baño o la cocina.\xa0'},
 {'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
  'pregunta': '¿Dónde puedo reportar una fuga de agua?',
  'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.'},
 {'id': 'edb019b4dbee72a582d41a5d60234bdd',
  'pregunta': '¿Qué hacer si tengo problemas con el suministro de agua potable?',
  'respuesta': 'En la Ciudad de México los encargados de proporcionar el servicio de agua potable, drenaje y tratamiento de aguas residuales a la población son los organismos operadores y las oficinas de agua potable de cada municipio, '},
 {'id': '5229ec6562607f2de830a7826a099964',
  'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?

In [52]:
question_vector_knn(ground_truth[10], 'pregunta_vector')

[{'id': 'edb019b4dbee72a582d41a5d60234bdd',
  'pregunta': '¿Qué hacer si tengo problemas con el suministro de agua potable?',
  'respuesta': 'En la Ciudad de México los encargados de proporcionar el servicio de agua potable, drenaje y tratamiento de aguas residuales a la población son los organismos operadores y las oficinas de agua potable de cada municipio, '},
 {'id': '5229ec6562607f2de830a7826a099964',
  'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?',
  'respuesta': 'Si no tienes fuga de agua en tu casa, puedes tomar la lectura del medidor, enviar una foto y comparar con el recibo anterior.\xa0Si hay discrepancia, puedes levantar un reporte.'},
 {'id': '9055d52c36234595f93430ca60610b82',
  'pregunta': '¿Cuánto tiempo tarda en repararse una fuga de agua?',
  'respuesta': 'Después de supervisar los materiales, maquinaria y personal, se comienzan los trabajos de reparación en un plazo de 48 horas.'},
 {'id': '34f49a09c09f373422893abead2bad4e',
  'pregunta'

In [53]:
question_vector_knn(ground_truth[10], 'preg_resp_vector')

[{'id': 'edb019b4dbee72a582d41a5d60234bdd',
  'pregunta': '¿Qué hacer si tengo problemas con el suministro de agua potable?',
  'respuesta': 'En la Ciudad de México los encargados de proporcionar el servicio de agua potable, drenaje y tratamiento de aguas residuales a la población son los organismos operadores y las oficinas de agua potable de cada municipio, '},
 {'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
  'pregunta': '¿Dónde puedo reportar una fuga de agua?',
  'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.'},
 {'id': 'b9e493259e0b9fb9f94e4f4d66ab8ded',
  'pregunta': '¿Qué debo hacer si veo una fuga de agua en mi casa?',
  'respuesta': 'Lo primero que debes hacer es cortar el suministro de agua, ya sea con el grifo de cierre o la válvula de cierre, que generalmente se encuentran en el baño o la cocina.\xa0'},
 {'id': '5229ec6562607f2de830a7826a099964',
  'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?

In [57]:
results = []
for field in ['pregunta_vector', 'respuesta_vector', 'preg_resp_vector']:
    res = evaluate(ground_truth, question_vector_knn, field)
    res['field'] = field
    results.append(res)

df_results = pd.DataFrame(results)


100%|██████████| 11/11 [00:01<00:00,  9.12it/s]


In [58]:
df_results

,hit_rate,mrr,field
0,1.000000,0.954545,pregunta_vector
1,0.818182,0.460606,respuesta_vector
2,1.000000,0.939394,preg_resp_vector
